# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [6]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Question 1: What is the sum of the outputs of the generator for limit = 5?
limit = 5
generator = square_root_generator(limit)

total = 0
for sqrt_value in generator:
  total = total + sqrt_value

print(f"Sum of the generator for limit = 5 : {total}")

Sum of the generator for limit = 5 : 8.382332347441762


In [7]:
# Question 2: What is the 13th number yielded
limit = 13
generator = square_root_generator(limit)

for i, sqrt_value in enumerate(generator):
  if i == limit - 1:
    print(f"The 13th number : {sqrt_value}")

The 13th number : 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [8]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [9]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [10]:
import dlt
import duckdb

data1 = []
for person in people_1():
  data1.append(person)

data2 = []
for person in people_2():
  data2.append(person)

# define the connection to load to.
pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

#conn.sql("SELECT * FROM people")
#conn.sql("drop table people_merge")

# run with append write disposition.
info = pipeline.run(data1,
										table_name="people",
										write_disposition="append")
# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708048965.0426068 is LOADED and contains no failed jobs


In [11]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# let's see the tables
print('Loaded tables: ')
display(conn.sql("show tables"))

people_table = conn.sql("SELECT * FROM people").df()
display(people_table)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708048965.0426068,Cd1X7O3LgsCONA
1,2,Person_2,27,City_A,1708048965.0426068,GfKqLS+1zLWRlQ
2,3,Person_3,28,City_A,1708048965.0426068,t4OzVotgJeKXoQ
3,4,Person_4,29,City_A,1708048965.0426068,aAScU2p+skzZ3A
4,5,Person_5,30,City_A,1708048965.0426068,PstDZU1nlojYuQ


In [12]:
# run with append write disposition.
info = pipeline.run(data2,
										table_name="people",
										write_disposition="append")
# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708048966.1685226 is LOADED and contains no failed jobs


In [13]:
people_table = conn.sql("SELECT * FROM people").df()
display(people_table)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708048965.0426068,Cd1X7O3LgsCONA,None
1,2,Person_2,27,City_A,1708048965.0426068,GfKqLS+1zLWRlQ,None
2,3,Person_3,28,City_A,1708048965.0426068,t4OzVotgJeKXoQ,None
3,4,Person_4,29,City_A,1708048965.0426068,aAScU2p+skzZ3A,None
4,5,Person_5,30,City_A,1708048965.0426068,PstDZU1nlojYuQ,None
5,3,Person_3,33,City_B,1708048966.1685226,CCulzvBlVBusCA,Job_3
6,4,Person_4,34,City_B,1708048966.1685226,GNaeHXKjJtx3UQ,Job_4
7,5,Person_5,35,City_B,1708048966.1685226,IuUQYLcXKGaGcw,Job_5
8,6,Person_6,36,City_B,1708048966.1685226,u20YxF62+81nUg,Job_6
9,7,Person_7,37,City_B,1708048966.1685226,Jns8CqgCAHTGfw,Job_7


In [14]:
# Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.
query = conn.sql("SELECT SUM(age) AS total_ages FROM people").df()
display(query)

,total_ages
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [15]:
# run with append write disposition.
info = pipeline.run(data1,
										table_name="people_merge",
										write_disposition="append")
# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708048966.913156 is LOADED and contains no failed jobs


In [16]:
# run with append write disposition.
info = pipeline.run(data2,
										table_name="people_merge",
										write_disposition="merge",
                    primary_key="id")
# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.45 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708048967.5867937 is LOADED and contains no failed jobs


In [17]:
people_merge_table = conn.sql("SELECT * FROM people_merge").df()
display(people_merge_table)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708048966.913156,qZ92o9F2y+DXwA,None
1,2,Person_2,27,City_A,1708048966.913156,2FSO6rRoJfU3og,None
2,5,Person_5,35,City_B,1708048967.5867937,LAwSh+1OMJ6+ZA,Job_5
3,4,Person_4,34,City_B,1708048967.5867937,kKgihkLAbPHlaw,Job_4
4,8,Person_8,38,City_B,1708048967.5867937,g0E4iyMhdYcQKw,Job_8
5,7,Person_7,37,City_B,1708048967.5867937,9fWztaVXQMOr8A,Job_7
6,3,Person_3,33,City_B,1708048967.5867937,09/7GqHGKhniXA,Job_3
7,6,Person_6,36,City_B,1708048967.5867937,sxccSahKFtBkfg,Job_6


In [18]:
# Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.
query = conn.sql("SELECT SUM(age) AS total_ages FROM people_merge").df()
display(query)

,total_ages
0,266.0


# Solution: First make sure that the following modules are installed:

In [19]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX